In [ ]:
# Verify Google gave you GPU runtime and switch using dropdown above if not
!nvidia-smi -L
!nvidia-smi

In [ ]:
# Set up colab paths
temp_colab_path = '/content/invokeai_temp_colab'
persist_colab_path = '/content/drive/MyDrive/invokeai_persist_colab'

# Set if instance should be temporary or persistent
default_colab_path = temp_colab_path

# Set up temporary storage if needed
if default_colab_path == temp_colab_path:
    import os
    if not os.path.exists(temp_colab_path):
      os.makedirs(temp_colab_path)

# Mount and set up Google drive if needed/isn't already
if default_colab_path == persist_colab_path:
  import os
  from google.colab import drive
  if not os.path.exists(persist_colab_path):
    drive.mount('/content/drive')
    if not os.path.exists(persist_colab_path):
      os.makedirs(persist_colab_path)

In [ ]:
# Sign up/accept terms/get tokens at https://huggingface.co/runwayml/stable-diffusion-v1-5 https://huggingface.co/runwayml/stable-diffusion-inpainting and https://ngrok.com/
huggingface_token = 'HF_TOKEN_HERE'
ngrok_token = 'NGROK_TOKEN_HERE'

In [ ]:
# Clone InvokeAI git repo
%cd {default_colab_path}
!rm -rf InvokeAI
!git clone https://github.com/invoke-ai/InvokeAI.git

In [ ]:
# Install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev -y

# Change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

# Verify python version
!python --version

# Update pip
%cd {default_colab_path}
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall
!python -m pip install --upgrade pip
!pip install --upgrade setuptools

# Handle colab dependencies
!ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

In [ ]:
# Link to select proper requirements file
%cd {default_colab_path}/InvokeAI
!ln -sf environments-and-requirements/requirements-lin-cuda.txt requirements.txt

In [ ]:
# Create virtual env and install all requirements
%cd {temp_colab_path}/InvokeAI
!pip install virtualenv
!virtualenv invokeai
!source invokeai/bin/activate; pip install --prefer-binary -r requirements.txt

In [ ]:
# Preconfigure and run setup/fetch script 
%cd {default_colab_path}/InvokeAI
!sed -i 's/enable_safety_checker\ =\ True/enable_safety_checker\ =\ False/' scripts/configure_invokeai.py
!source invokeai/bin/activate; echo -e 'import huggingface_hub\nhuggingface_hub.login("{huggingface_token}")' | python
!source invokeai/bin/activate; python3 scripts/configure_invokeai.py --yes

In [ ]:
# Custom variants of the stable diffusion model can be created through meeans such as https://colab.research.google.com/github/ShivamShrirao/diffusers/blob/main/examples/dreambooth/DreamBooth_Stable_Diffusion.ipynb
# The below code is an example of how to import and configure models through script
# Import custom models
#%cd {default_colab_path}/InvokeAI
#!cp -r /content/drive/MyDrive/stable_diffusion_weights/* models

# Configure custom models
#%cd {default_colab_path}/InvokeAI
#import os
#import shutil
#if not os.path.exists('configs/original-models.yaml'):
#  shutil.copy('configs/models.yaml', 'configs/original-models.yaml')
#!cp configs/original-models.yaml configs/models.yaml
#!cat /content/drive/MyDrive/stable_diffusion_weights/*/*.yaml >> configs/models.yaml

# Set chosen model to default
#!sed -i 's/default:\ true/default:\ false/' configs/models.yaml
#!sed -i '15s/default:\ false/default:\ true/' configs/models.yaml # vanilla
#!sed -i '38s/default:\ false/default:\ true/' configs/models.yaml # tftpismyfavport
#!sed -i '30s/default:\ false/default:\ true/' configs/models.yaml # jenkinsmichpa


In [ ]:
# Install and run NGROK proxy
%cd {default_colab_path}
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!chmod +x ngrok
!pkill -9 -f '.*ngrok.*'
!./ngrok authtoken {ngrok_token}
get_ipython().system_raw('./ngrok http 9090 &')
!sleep 10

# Print address of proxy site to access instance
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# Start site
%cd {default_colab_path}/InvokeAI
!export INVOKEAI_ROOT=/content/invokeai_temp_colab/InvokeAIStorage
!source invokeai/bin/activate; python scripts/invoke.py --web --host 0.0.0.0